In [1]:
import pandas as pd

In [ ]:
# We're off to a pretty bad start. I have no idea where this data comes from. I assume this is data that you somehow created by transforming
# data from the 3 datasets I got from running the .py files in this project, but I really have no idea how this happened or how I could recreate
# it so I'm going to be doing a lot of shooting in the dark from here on out. 

data = pd.read_csv("data/SalaryData.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/SalaryData.csv'

# Cleaning 

In [ ]:
# I'll start by saying that I believe your target variable aka 'y' aka the dependent variable aka the metric you would like to predict is 
# discrepancy ratio, based on what I can deduce by reading through this file. Anything I write from here on out is based on this assumption which 
# at this point is just that. From what I've put together at this point my main concern with this as far as manifesting this into an ML project is 
# figuring out what x (x is almost always a collection of things,) aka the predictor variables aka independent variables might be. Let's imagine 
# we're doing multiple regression, we need a formula along the lines y = m1x1 + m2x2 + m3x3 + mnxn + b, with b being the y intercept and the xs 
# being our predictor variables (n in this instance implies an unknown number of predictors.) So basically our concern at this point is finding 
# variables correlated with y which can be used to predict it.

cols_to_drop = ['Employee_Name', 'jobTitle', 'Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
data = data.drop(columns=cols_to_drop)

# I'm interested in this incentive allowance metric and where it came from. It might bear fruit using it as a predictor if we can better
# understand it's role and correlation to discrepancy/discrepancy ratio.

data['Salary_Total'] = data['Annual_Rate'] + data['Incentive_Allowance']

# I think both of these drops are misguided in the quest for predictor variables frankly. Though we have above created a metric used to
# represent them we are watering down their value as predictors by creating the above aggregate.

final_drop = ['Annual_Rate', 'Incentive_Allowance']
data = data.drop(columns=final_drop)

# I will note when using .fillna() it is always expected that you would have some rationale for your strategy. I think it's probably pretty
# straightforward here, but do anticipate this being questioned. Many would likely .fillna() on a column by column basis filling in the median
# value relative to the column (presuming 'continuous' or numeric values which could hypothetically range from 0 to infinity.) I'll also note
# there is a special kind of regression known as 'poisson' regression which might be worth considering for your purposes...It is particularly 
# useful when the target variable is known to not be truly continuous (ala -infinity to infinity) but known to fall within a range 
# characterized by 0 to 1 as this is how it works under the hood though in practice this can be used for any range. Poisson is particuarly 
# powerful when working with probabilities.
# https://en.wikipedia.org/wiki/Poisson_regression

data = data.fillna(0)
data['Department'] = data['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')

In [ ]:
# This would be an insanely small dataset to do any kind of ML with both in rows and columns but most concerningly in columns. It makes a nice
# aggregate for example for a PowerPoint slide, but would not be ideal to feed to a model.

data.head()

,CalYear,Department,YTD_Total,Salary_Total
0,2023,OMB Finance,1600.00,5200.00
1,2023,Louisville Free Public Library,30657.64,30960.00
2,2023,Louisville Metro Police Department,41457.53,41457.52
3,2023,Louisville Metro Police Department,19823.87,16912.00
4,2023,Louisville Metro Police Department,58513.88,61796.80


In [35]:
# final_salary = data.groupby('Department')[['YTD_Total', 'Salary_Total']].sum().reset_index()
final_salary = data.groupby(['CalYear', 'Department'])[['YTD_Total', 'Salary_Total']].sum().reset_index()


In [36]:
final_salary.head(2)

,CalYear,Department,YTD_Total,Salary_Total
0,2019,Air Pollution Control,3161000.01,3357059.36
1,2019,Belle of Louisville,1288272.26,2255142.24


In [37]:
final_salary.head(2)

,CalYear,Department,YTD_Total,Salary_Total
0,2019,Air Pollution Control,3161000.01,3357059.36
1,2019,Belle of Louisville,1288272.26,2255142.24


# Lmpd general view

In [38]:
lmpd = final_salary[final_salary['Department'] == 'Louisville Metro Police Department'].copy()
lmpd = lmpd[lmpd['CalYear'] != 2024].copy()
lmpd

,CalYear,Department,YTD_Total,Salary_Total
19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07
56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08
93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08
131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08
176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08


In [39]:
# lmpd.to_csv('data/test.csv')

In [40]:
lmpd = pd.read_csv('data/test.csv')

In [ ]:

# I'm assuming unnamed is row #s. This obviously has to go. 

lmpd

,Unnamed: 0,CalYear,Department,YTD_Total,Salary_Total
0,19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07
1,56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08
2,93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08
3,131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08
4,176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08


In [42]:
lmpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    5 non-null      int64  
 1   CalYear       5 non-null      int64  
 2   Department    5 non-null      object 
 3   YTD_Total     5 non-null      float64
 4   Salary_Total  5 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 332.0+ bytes


In [ ]:
# This kind of 'feature engineering' is insanely useful. We still don't have much in the way of independent variables though. Since we have 
# engineered our dependent from two other columns we know these will have high correlation and have removed their usefulness in prediction...
# as we know their relationship to the independent in philosophy should be largely 1:1. 

# Using `.loc` ensures that we are modifying the DataFrame in place, 
# rather than working on a temporary slice of the data.
lmpd['Discrepancy'] = lmpd['YTD_Total'] - lmpd['Salary_Total']
lmpd['Discrepancy_Ratio'] = lmpd['Discrepancy'] / lmpd['Salary_Total']

lmpd

,Unnamed: 0,CalYear,Department,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
0,19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07,3590832.44,0.036409
1,56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08,2092070.71,0.020104
2,93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08,1290365.61,0.012639
3,131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08,17218319.86,0.163687
4,176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08,4122435.66,0.034882


In [44]:
# pd.set_option('display.float_format', '{:,.2f}'.format)


In [45]:
lmpd

,Unnamed: 0,CalYear,Department,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
0,19,2019,Louisville Metro Police Department,1.022160e+08,9.862518e+07,3590832.44,0.036409
1,56,2020,Louisville Metro Police Department,1.061527e+08,1.040606e+08,2092070.71,0.020104
2,93,2021,Louisville Metro Police Department,1.033841e+08,1.020937e+08,1290365.61,0.012639
3,131,2022,Louisville Metro Police Department,1.224085e+08,1.051902e+08,17218319.86,0.163687
4,176,2023,Louisville Metro Police Department,1.223053e+08,1.181829e+08,4122435.66,0.034882


In business, a discrepancy ratio between expected salary spend (Salary_Total) and actual salary spend (YTD_Total) typically falls within 0 to 5% as an acceptable range. A discrepancy within this range usually indicates minor variations in salary spending, such as unexpected overtime, temporary staffing, or adjustments to salary levels, which are often manageable within the overall budget. A discrepancy ratio of more than 10% would generally raise concerns, signaling potential overspending on salaries, and would require a deeper investigation into the causes, such as unplanned hiring, excessive overtime, or compensation adjustments.

For the Louisville Metro Police Department, the discrepancy ratios from 2019 to 2023 show a range from 0.01 to 0.16. The ratios in 2019, 2020, 2021, and 2023 are all within a 0.01 to 0.04 range, which suggests a relatively small and manageable variance in salary spending. However, the discrepancy ratio for 2022 is significantly higher at 0.16, indicating a 16% overspend on salaries compared to the planned budget for that year. This could point to specific factors in 2022 that led to a sharp increase in salary expenses, such as a one-time hiring initiative, increased overtime costs, or other unforeseen budgetary pressures.

# LMPD indv level

In [46]:
indv = pd.read_csv('data/SalaryData.csv')

In [ ]:
# This individual level of analysis is definitely going to yield better results for ML. If your thought process was to have everything above to
# illustrate the question you are trying to answer then you can justify that but perhaps this notebook isn't the best place for it. 

cols_to_drop = ['Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
indv = indv.drop(columns=cols_to_drop)

# As noted above I think 'incentive allowance' may be the independent with the strongest explanatory power and I'm not sure I'd agg it away.

indv['Salary_Total'] = indv['Annual_Rate'] + indv['Incentive_Allowance']
final_drop = ['Annual_Rate', 'Incentive_Allowance']
indv = indv.drop(columns=final_drop)
indv = indv.fillna(0)
indv['Department'] = indv['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')

# I had mentioned previously when evaluating models you typically create a 'test' and 'training' set. Essentially you remove a subset of your 
# data to be able to test your model against, training on some data and using the test set to evaluate the effectiveness of your training. Is
# the idea here to train on previous years to test against 2024? Or was it removed simply because it is at this point incomplete? It may be worth
# using 2024 as your test set for your purposes. As general background I typically use a 80/20 train/test set removing randomly meaning that 20% 
# that ends up in test would be records from across years, but there is no reason it has to be this way. However when working with time series it 
# is common that comparing a 2019 record with a 2024 record is oftentimes not comparing apples to apples as there are usually trackable changes in
# a 'normal state' over a 5 year period and you can be losing some of the insight they could yield. If you can identify them these metrics
# which cause deviation from a 'normal state' over time tend to be great independent variables. This likely does not exist in your raw now and 
# would have to somehow be brought in

indv = indv[indv['CalYear'] != 2024].copy()

In [48]:
# indv.head(2)

In [49]:
indv = indv[indv['Department'] == 'Louisville Metro Police Department'].copy()
# indv.head(2)

In [50]:
indv['Discrepancy'] = indv['YTD_Total'] - indv['Salary_Total']
indv['Discrepancy_Ratio'] = indv['Discrepancy'] / indv['Salary_Total']


In [51]:
indv.head(2)

,CalYear,Employee_Name,Department,jobTitle,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
2,2023,"Bratcher, Elaine",Louisville Metro Police Department,Clerk II-Police,41457.53,41457.52,0.01,2.412108e-07
3,2023,"Jackson, Nila Marie",Louisville Metro Police Department,Traffic Guard I,19823.87,16912.00,2911.87,1.721777e-01


Top 10 Employees with the Highest Discrepancies

In [ ]:
# This kind of exploration can be useful to identify independent variables and is typically part of EDA or exploratory data analysis. You explore
# for a couple of reasons but identifying independents is a pretty highly valued one. If you believe there to be value in anything that you have
# uncovered in this EDA then you want to test correlation with the target variable to assess if there really could be explanatory power. As a 
# random hint I will say it is common in ML to use 'dummy variables' (which pandas has a built in method for) to allow for categorical variables
# (like job role) to be used for correlation. This works under the hood basically like creating a boolean (is sergeant) metric for each role 
# which will contain either a 1 or 0 as a value. 
# 
# https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html 

def top_10_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Sort by 'Discrepancy' in descending order to get the top 10 employees
    top_10_employees = indv_year.sort_values(by='Discrepancy', ascending=False).head(10)
    
    # Return the top 10 employees with their discrepancies
    return top_10_employees[['CalYear', 'Employee_Name', 'jobTitle', 'Discrepancy', 'YTD_Total', 'Salary_Total']]



In [53]:
# Example usage:
top_emp_dis_2019 = top_10_discrepancy(indv, 2019)
top_emp_dis_2019

,CalYear,Employee_Name,jobTitle,Discrepancy,YTD_Total,Salary_Total
14865,2019,"Fey, Ronald",Police Sergeant,48026.85,126355.33,78328.48
15520,2019,"Keller, Matthew",Police Sergeant,47451.70,129405.14,81953.44
14923,2019,"Cohn, Micah",Police Officer,46884.58,119415.06,72530.48
15551,2019,"Patterson, William",Police Sergeant,41370.84,124059.96,82689.12
18073,2019,"Isham, Vernon",Storage Equipment Operator5day,40811.59,84371.99,43560.40
16120,2019,"Bassler, David",Police Officer,38621.32,103770.84,65149.52
16995,2019,"O'Daniel, Timothy",Police Officer,34829.01,99455.41,64626.40
16439,2019,"Gillis, Bryan",Police Officer,34154.93,100037.41,65882.48
16128,2019,"Vance, Jason",Police Sergeant,32688.77,110145.81,77457.04
14368,2019,"King, Michael",Police Sergeant,32284.70,117777.50,85492.80


In [54]:
top_emp_dis_2020 = top_10_discrepancy(indv, 2020)
top_emp_dis_2020

,CalYear,Employee_Name,jobTitle,Discrepancy,YTD_Total,Salary_Total
22390,2020,"Keller, Matthew",Police Sergeant,69394.43,162998.75,93604.32
22193,2020,"Smith, Jeremy",Police Sergeant,62327.48,155383.88,93056.40
21894,2020,"Fey, Ronald",Police Sergeant,61428.87,151054.15,89625.28
22207,2020,"Szydlowski, Victor",Police Sergeant,59551.93,151002.81,91450.88
21580,2020,"Watkins, Kelvin",Police Officer,57862.27,137577.79,79715.52
21300,2020,"Miller, Russell",Police Lieutenant,54094.56,155163.36,101068.80
21296,2020,"Hume, Jill",Police Lieutenant,53387.49,154541.49,101154.00
21266,2020,"Burbrink, Donald",Police Lieutenant,51273.52,152204.80,100931.28
21410,2020,"Rivera, Arnold",Police Lieutenant,49459.40,150423.80,100964.40
22908,2020,"Claxon, Wesley",Police Officer,48626.56,126546.00,77919.44


In [55]:
top_emp_dis_2022 = top_10_discrepancy(indv, 2022)
top_emp_dis_2022

,CalYear,Employee_Name,jobTitle,Discrepancy,YTD_Total,Salary_Total
28061,2022,"Fey, Ronald W",Police Sergeant,99793.00,199973.34,100180.34
30909,2022,"Dahlgren, Patrick Barthold",Police Officer,92495.11,171864.72,79369.61
31609,2022,"Crawford, Montrell",Police Officer,91343.61,169561.49,78217.88
28101,2022,"Keller, Matthew J",Police Lieutenant,86355.73,212051.00,125695.27
30288,2022,"Koestel, Matthew Robert",Police Officer,84322.63,165746.73,81424.10
29285,2022,"Flannery, Aaron Jerome",Police Officer,84290.25,164265.53,79975.28
29549,2022,"Mattiche, Youssef",Police Officer,80721.37,163813.87,83092.50
28102,2022,"Neal, Paul Eugene",Police Sergeant,79044.77,184108.74,105063.97
31859,2022,"Capito, Jacob Paul",Police Officer,78714.69,155655.01,76940.32
28941,2022,"Sears, Shelby Edward Vanwinkle",Police Sergeant,74790.00,171524.56,96734.56


Job Title with the Largest Individual Discrepancy by Year:

In [56]:
# Group by CalYear and jobTitle to find the row with the largest discrepancy per year
largest_discrepancy_by_year = indv.loc[indv.groupby('CalYear')['Discrepancy'].idxmax()]

# Display the job title and the corresponding discrepancy
largest_discrepancy_by_year[['CalYear', 'jobTitle', 'Discrepancy']]

,CalYear,jobTitle,Discrepancy
14865,2019,Police Sergeant,48026.85
22390,2020,Police Sergeant,69394.43
10694,2021,Police Officer,137606.36
28061,2022,Police Sergeant,99793.00
1806,2023,Police Sergeant,116884.19


top 10 job titles by avg discrepancy by year

In [57]:
def top_10_job_titles_by_avg_discrepancy(indv, enter_year):
    # Filter the DataFrame for the given year
    indv_year = indv[indv['CalYear'] == enter_year]
    
    # Group by 'jobTitle' and calculate the average 'Discrepancy' for each job title
    avg_discrepancy_by_job = indv_year.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort by the average 'Discrepancy' in descending order to get the top 10 job titles
    top_10_job_titles = avg_discrepancy_by_job.sort_values(ascending=False).head(10)
    
    # Return the relevant columns (jobTitle and average Discrepancy)
    return top_10_job_titles.reset_index()[['jobTitle', 'Discrepancy']]

In [58]:
# Example usage:
top_10_job_titles_by_avg_discrepancy_2023 = top_10_job_titles_by_avg_discrepancy(indv, 2023)
top_10_job_titles_by_avg_discrepancy_2023

,jobTitle,Discrepancy
0,Tow-In Equipment Operator 6day,53743.320000
1,Storage Equipment Operator 5 Day,24019.662000
2,Information Process Tech II,19541.015000
3,Traffic Guard Supervisor,16161.520000
4,Information Systems Supervisor,13577.490000
5,Group Violence Victimology Specialist,12925.525000
6,Non-Sworn Rotorcraft Pilot,12502.400000
7,Technology Program Manager,12205.090000
8,Police Lieutenant,11157.732969
9,Police Sergeant,10827.717278


Average Discrepancy per Job Title:

In [59]:
def avg_discrepancy_by_job(indv):
    # Group by 'jobTitle' and calculate the average discrepancy (YTD_Total - Salary_Total)
    avg_discrepancy_per_job = indv.groupby('jobTitle')['Discrepancy'].mean()
    
    # Sort the result by average discrepancy in descending order
    avg_discrepancy_per_job_sorted = avg_discrepancy_per_job.sort_values(ascending=False)
    
    # Return the sorted average discrepancies
    return avg_discrepancy_per_job_sorted

In [60]:
# Example usage:
avg_discrepancy = avg_discrepancy_by_job(indv)
avg_discrepancy

jobTitle
Special Assistant- PT               32913.850000
Special Assistant - PT              32479.040000
Storage Equipment Operator 5 Day    24131.235000
Information Process Tech II         17177.815000
Tow-In Equipment Operator 6day      16195.246000
                                        ...     
Firearms & Toolmark Examiner       -34152.880000
Training Academy Instructor        -35417.588571
Curriculum Development Coord       -39393.660000
Project Manager II                 -49539.000000
Police Chief                       -69311.240000
Name: Discrepancy, Length: 113, dtype: float64